# LangChain: Evaluation

Watch:

https://learn.deeplearning.ai/langchain/lesson/6/evaluation 

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
llm_model: str = "gpt-3.5-turbo-1106"

## Create our QandA application

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
from langchain.schema import Document

file: str = 'OutdoorClothingCatalog_1000.csv'
loader: CSVLoader = CSVLoader(file_path=file)
data: list[Document] = loader.load()

In [ ]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

index: VectorStoreIndexWrapper = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
from langchain.chains.retrieval_qa.base import BaseRetrievalQA

llm: ChatOpenAI = ChatOpenAI(temperature = 0.0, model=llm_model)
qa: BaseRetrievalQA = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [ ]:
data[10]

In [ ]:
data[11]

### Hard-coded examples

In [ ]:
examples: list[dict[str, str]] = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples